In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch diffusers opencv-python Pillow transformers scipy accelerate rembg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00


In [3]:
import cv2
from PIL import Image
import torch
from diffusers import StableDiffusionXLPipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
def generate_image_from_prompt(prompt):
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
    )
    pipe = pipe.to("cuda")
    pipe.batch_size = 1

    image = pipe(prompt).images[0]
    return image

In [5]:
def extract_alpha_channel(img):
    if img.mode != 'RGBA':
        img = img.convert('RGBA')
    r, g, b, a = img.split()
    alpha_array = np.array(a)
    return alpha_array

In [6]:
def get_largest_contour(alpha_channel):
    # Smoothing using GaussianBLur
    smoothed = cv2.GaussianBlur(alpha_channel, (15, 15), 0)
    contours_smoothed = cv2.findContours(
        smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_smoothed = contours_smoothed[0] if len(
        contours_smoothed) == 2 else contours_smoothed[1]
    big_contours_smoothed = max(contours_smoothed, key=cv2.contourArea)
    peri = cv2.arcLength(big_contours_smoothed, True)
    return cv2.approxPolyDP(big_contours_smoothed, 0.001 * peri, True)

In [7]:
def draw_filled_contour_on_black_background(big_contour, shape):
    contour_img = np.zeros(shape)
    cv2.drawContours(contour_img, [big_contour], 0, 255, -1)
    return contour_img    

In [8]:
def apply_dilation(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30, 30))
    return cv2.morphologyEx(img, cv2.MORPH_DILATE, kernel)

In [9]:
def apply_overlays(canvas, img, dilate):
    if img.shape[2] == 3:  # Convert RGB → RGBA
        img = np.dstack([img, np.ones(img.shape[:2], dtype=np.uint8) * 255])
    alpha = img[:, :, 3]
    alpha = np.expand_dims(alpha, 2)
    alpha = np.repeat(alpha, 3, 2)
    alpha = alpha / 255

    canvas[dilate == 255] = (255, 255, 255, 255)
    canvas[:, :, 0:3] = canvas[:, :, 0:3] * (1 - alpha) + alpha * img[:, :, 0:3]

    return canvas

In [10]:
def create_sticker(img):
    img_array = np.array(img)
    alpha = extract_alpha_channel(img)
    big_contour = get_largest_contour(alpha)
    contour_img = draw_filled_contour_on_black_background(
        big_contour, alpha.shape)
    dilate = apply_dilation(contour_img)

    canvas = np.zeros((img_array.shape[0], img_array.shape[1], 4),dtype=np.uint8)
    canvas = apply_overlays(canvas, img_array, dilate)

    return canvas.astype(np.uint8)

In [23]:
def show_sticker_from_image(image,text):
    sticker = Image.fromarray(image)
    sticker.save(f"{text}.png")
    print(f"Sticker saved as {text}.png")

In [24]:
def main():
    prompt = input("Enter prompt:")
    image = generate_image_from_prompt(prompt)
    sticker = create_sticker(image)
    show_sticker_from_image(sticker, prompt)
    

In [25]:
main()

Enter prompt: baby in dreadlocks


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Sticker saved as baby in dreadlocks.png


In [19]:
main()

Enter prompt: Thor with an hammer.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Sticker saved as sticker.png
